In [1]:
### Assigned by ###
# Gal Israeli #####
# Yulia Kuderko ###
# Ram Michaeli ####
###################

import pandas as pd
import time
from datetime import datetime as dt, timedelta
import json
import requests
import re
import matplotlib as plt
from bs4 import BeautifulSoup
import numpy as np
import os
import sys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium import webdriver
from requests import get
import seaborn as sns

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#!pip install selenium

# <font color='#38ADC8'><u>First Glimpse at Our Database</u></font>

In [2]:
df=pd.read_csv("Volcano_Final.csv")
df

,volcanoName,eruptionCertainty,startDate,maxVEI,activityArea,evidenceMethod,stopDate,latitude,longitude,volcanoHeight,country,primaryVolcanoType,population5km,population10km,population30km,population100km
0,Ambae,Confirmed Eruption,2021 Dec 5,--,NaN,Historical Observations,2021 Dec 9,-15.389,167.835,1496 m,Vanuatu,Shield,73,"1,279","4,326","73,308"
1,Pinatubo,Confirmed Eruption,2021 Nov 30,--,NaN,Historical Observations,2021 Nov 30,15.130,120.350,1486 m,Philippines,Stratovolcano,725,"3,808","1,148,684","21,875,048"
2,Iliwerung,Confirmed Eruption,2021 Nov 28,--,NaN,Historical Observations,2021 Nov 29,-8.532,123.573,583 m,Indonesia,Complex,968,"4,318","79,569","665,081"
3,Taal,Confirmed Eruption,2021 Nov 15,--,NaN,Historical Observations,2021 Nov 22,14.002,120.993,311 m,Philippines,Caldera,"717,090","717,090","2,380,326","24,814,047"
4,Turrialba,Confirmed Eruption,2021 Nov 3,--,NaN,Historical Observations,2021 Nov 23,10.025,-83.767,3340 m,Costa Rica,Stratovolcano,"1,891","6,867","672,428","3,390,544"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,"Yojoa, Lago",Confirmed Eruption,1073 BCE,--,NaN,Radiocarbon (corrected),Unknown,14.964,-87.983,1060 m,Honduras,Volcanic field,"123,374","123,374","279,991","3,336,190"
11218,Quetrupillan,Confirmed Eruption,1345 BCE,3,Quet2,Radiocarbon (corrected),Unknown,-39.496,-71.722,2360 m,Chile,Stratovolcano,129,518,"11,191","269,087"
11219,Saunders,Confirmed Eruption,,0,NaN,Unknown,Unknown,-57.800,-26.483,843 m,United Kingdom,Stratovolcano,0,0,0,0
11220,"Fournaise, Piton de la",Confirmed Eruption,,--,NaN,Unknown,Unknown,-21.244,55.708,2632 m,France,Shield,"55,859","55,859","246,792","717,820"


In [3]:
df.columns

Index(['volcanoName', 'eruptionCertainty', 'startDate', 'maxVEI',
       'activityArea', 'evidenceMethod', 'stopDate', 'latitude', 'longitude',
       'volcanoHeight', 'country', 'primaryVolcanoType', 'population5km',
       'population10km', 'population30km', 'population100km'],
      dtype='object')

In [4]:
df.shape

(11222, 16)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11222 entries, 0 to 11221
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   volcanoName         11222 non-null  object 
 1   eruptionCertainty   11222 non-null  object 
 2   startDate           11222 non-null  object 
 3   maxVEI              11222 non-null  object 
 4   activityArea        5193 non-null   object 
 5   evidenceMethod      11145 non-null  object 
 6   stopDate            11145 non-null  object 
 7   latitude            11145 non-null  float64
 8   longitude           11145 non-null  float64
 9   volcanoHeight       11145 non-null  object 
 10  country             11145 non-null  object 
 11  primaryVolcanoType  11145 non-null  object 
 12  population5km       11114 non-null  object 
 13  population10km      11100 non-null  object 
 14  population30km      11100 non-null  object 
 15  population100km     11100 non-null  object 
dtypes: f

In [6]:
df.describe(include='all')

,volcanoName,eruptionCertainty,startDate,maxVEI,activityArea,evidenceMethod,stopDate,latitude,longitude,volcanoHeight,country,primaryVolcanoType,population5km,population10km,population30km,population100km
count,11222,11222,11222,11222,5193,11145,11145,11145.000000,11145.000000,11145,11145,11145,11114,11100,11100,11100
unique,914,3,6935,9,3268,22,4143,NaN,NaN,815,81,28,516,609,718,817
top,Etna,Confirmed Eruption,0050 BCE,2,Naka-dake,Historical Observations,Unknown,NaN,NaN,3320 m,Japan,Stratovolcano,0,0,0,0
freq,241,9944,33,4020,171,6397,5813,NaN,NaN,241,1725,5254,2848,2311,1079,505
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.962395,31.936626,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.788674,115.613025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-77.530000,-179.970000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.137000,-77.870000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.222000,55.708000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.821000,139.394000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# <font color='#38ADC8'><u>Cleaning</u></font>

###### We will check each column to see if it needs cleaning


### <u>Volcano Name</u>

###### We can see we have multiple volcanoes with the same name ('Unnamed') , but they are different

In [7]:
df[df['volcanoName'] == 'Unnamed']

,volcanoName,eruptionCertainty,startDate,maxVEI,activityArea,evidenceMethod,stopDate,latitude,longitude,volcanoHeight,country,primaryVolcanoType,population5km,population10km,population30km,population100km
153,Unnamed,Confirmed Eruption,2017 Jan 23,0,NaN,Historical Observations,2017 Jan 31,-20.852,-175.550,-296 m,Tonga,Submarine,0,0,0,"88,571"
817,Unnamed,Confirmed Eruption,2001 Sep 27,2,NaN,Historical Observations,2001 Sep 28 (?),-18.325,-174.365,-40 m,Tonga,Submarine,0,0,20,"18,153"
936,Unnamed,Confirmed Eruption,1999 Jan 8,1,NaN,Historical Observations,1999 Jan 14 (?),-20.852,-175.550,-296 m,Tonga,Submarine,0,0,0,"88,571"
1973,Unnamed,Confirmed Eruption,1972 Apr 29,0,NaN,Hydrophonic,1972 Apr 30,46.470,151.280,-502 m,Russia,Submarine,0,0,0,578
2064,Unnamed,Uncertain Eruption,1970 Jan 3,0,NaN,,[ 1970 Jan 3 ],-20.852,-175.550,-296 m,Tonga,Submarine,0,0,0,"88,571"
2175,Unnamed,Uncertain Eruption,1967 Apr 26,--,17.5 km NW of Iturup Island,,[ Unknown ],45.022,147.019,-195 m,Japan - administered by Russia,Submarine,0,0,693,"5,920"
3388,Unnamed,Confirmed Eruption,1932 Dec 1,0,NaN,Historical Observations,Unknown,-21.338,-175.650,-68 m,Tonga,Submarine,0,0,0,"88,771"
3704,Unnamed,Confirmed Eruption,1923 Jul 1,0,NaN,Historical Observations,Unknown,-20.852,-175.550,-296 m,Tonga,Submarine,0,0,0,"88,571"
4040,Unnamed,Confirmed Eruption,1911 Aug,0,NaN,Historical Observations,Unknown,-20.852,-175.550,-296 m,Tonga,Submarine,0,0,0,"88,571"
4173,Unnamed,Confirmed Eruption,1907 Jul,0,NaN,Historical Observations,Unknown,-21.338,-175.650,-68 m,Tonga,Submarine,0,0,0,"88,771"


In [8]:
# Here we create df2 , which is all the "unnamed" volcanoes in 1 small data frame.
# Each unnamed volcano may represent a different volcano , so we categorized them by number (unnamed 1 to 6)


df2 = df[df['volcanoName'] == 'Unnamed'].copy().sort_values('latitude')


index=0
counter=1
temp=None
for i, j in df2.iterrows():
    if(index==0):
        df2['volcanoName'].iloc[0] = df2['volcanoName'].iloc[0].replace('Unnamed','Unnamed '+str(counter))
    else:
        if (j['latitude']==temp['latitude'] and j['longitude']==temp['longitude']):
            df2['volcanoName'].iloc[index] = df2['volcanoName'].iloc[index].replace('Unnamed','Unnamed '+str(counter))
        else:
            counter+=1
            df2['volcanoName'].iloc[index] = df2['volcanoName'].iloc[index].replace('Unnamed','Unnamed '+str(counter))
    index+=1
    temp = j
    
# Here we replace the unnamed volcanoes in df to categorized unnamed volcanoes (1-6) using df2 we created above

df2_index = 0
for i,j in df2.iterrows():
    df['volcanoName'].iloc[i] = df['volcanoName'].iloc[i].replace('Unnamed',df2['volcanoName'].iloc[df2_index])
    print(df['volcanoName'].iloc[i])
    df2_index+=1


Unnamed 1
Unnamed 1
Unnamed 2
Unnamed 2
Unnamed 2
Unnamed 2
Unnamed 2
Unnamed 3
Unnamed 4
Unnamed 4
Unnamed 4
Unnamed 5
Unnamed 6


### <u> Eruption Certainty</u>

In [9]:
df.eruptionCertainty.value_counts().to_frame()

# We can see that we have no cleaning to do here

,eruptionCertainty
Confirmed Eruption,9944
Uncertain Eruption,1111
Discredited Eruption,167


### <u> Start Date</u>

In [10]:
df.startDate.value_counts().to_frame()

# All the dates seem to be okay

,startDate
0050 BCE,33
2050 BCE,33
1550,29
0950,27
0550 BCE,27
...,...
1954 May 18,1
1954 May 26,1
1954 May 28,1
1954 May 31,1


### <u> max VEI</u>

In [11]:
df.maxVEI.value_counts().to_frame()

# We can see that 2902 rows have '--' for their maxVEI , so we will change it to NaN
# for easier processing later

,maxVEI
2,4020
--,2902
1,1385
3,1159
0,1009
4,510
5,177
6,52
7,8


In [12]:
df=df[~df.isin(["--"])]

df.maxVEI.value_counts().to_frame()

# We can see we no longer have "--" in our maxVEI column.

,maxVEI
2,4020
1,1385
3,1159
0,1009
4,510
5,177
6,52
7,8


### <u> Activity Area</u>

In [13]:
rowsWithArea = df.activityArea.value_counts().to_frame().sum()
totalRows = df.shape[0]

print(str((1- round((rowsWithArea/totalRows)[0],2)))+"% volcanoes don't have activityArea value!")
# Over 50% of the data don't have "activityArea" so we will remove this column

df.drop('activityArea', axis=1, inplace=True)

'activityArea' not in df.columns

0.54% volcanoes don't have activityArea value!


True

### <u> Evidence Method</u>

In [14]:
df.evidenceMethod.value_counts().to_frame()

# We can see we have some values which are not NaN but seem like they should be NaN..
# We can also see we have "Unknown" and "Uncertain" values , we will replace them with NaN

# Lets figure out what it is , and replace it with NaN value

,evidenceMethod
Historical Observations,6397
,1278
Tephrochronology,1037
Radiocarbon (uncorrected),982
Radiocarbon (corrected),854
Varve Count,187
Hydrophonic,67
Magnetism,58
Unknown,50
Ice Core,45


In [15]:
# Lets look at a row which has no evidence method and see what is the string inside 

df[df['volcanoName']=='Cleveland'].iloc[1]['evidenceMethod']

'\xa0'

In [16]:
# \xa0 is that string! lets replace it (along with 'Unknown' and 'Uncertain')
# with an actual NaN value

df=df[~df.isin(["\xa0"])]
df=df[~df.isin(["Unknown"])]
df=df[~df.isin(["Uncertain"])]
df.evidenceMethod.value_counts().to_frame()

,evidenceMethod
Historical Observations,6397
Tephrochronology,1037
Radiocarbon (uncorrected),982
Radiocarbon (corrected),854
Varve Count,187
Hydrophonic,67
Magnetism,58
Ice Core,45
Anthropology,35
Uranium-series,31


### <u> Stop Date</u>

In [17]:
rowsWithStopDate = df.stopDate.value_counts().to_frame().sum()
totalRows = df.shape[0]

print(str((1- round((rowsWithStopDate/totalRows)[0],2)))+"% volcanoes don't have stopDate value!")
# Over 50% of the data don't have "stopDate" so we will remove this column

df.drop('stopDate', axis=1, inplace=True)

'stopDate' not in df.columns

0.52% volcanoes don't have stopDate value!


True

### <u> Latitude</u>

In [18]:
df.latitude.value_counts().to_frame()

# We can see all the latitude values seem to be okay
# We will check this later when we will convert the column type from object to int64
# If non-numeric values exist , it shouldn't work , and we will come back to fix it

,latitude
37.748,241
-21.244,197
32.884,187
-39.420,164
36.406,147
...,...
-17.874,1
44.411,1
25.500,1
3.248,1


### <u> Longitude</u>

In [19]:
df.longitude.value_counts().to_frame()

# Same thing goes here


,longitude
14.999,241
55.708,197
131.104,187
-71.930,164
138.523,147
...,...
113.570,1
-178.006,1
-90.743,1
-89.780,1


### <u> Volcano Height</u>

In [20]:
df.volcanoHeight.value_counts().to_frame()

# We can see all the volcano height values seem to be okay...
# (Btw : negative values mean underwater volcanoes!)

,volcanoHeight
3320 m,241
2632 m,197
1490 m,197
1592 m,187
2847 m,164
...,...
887 m,1
1068 m,1
1138 m,1
1028 m,1


### <u> Country</u>

In [21]:
df.country.value_counts().to_frame()

# We can see all the country values seem to be okay

,country
Japan,1725
Indonesia,1467
United States,1326
Russia,974
Chile,580
...,...
Vietnam,1
Ethiopia-Djibouti,1
Mongolia,1
Colombia-Ecuador,1


### <u> Primary Volcano Type</u>

In [22]:
df.primaryVolcanoType.value_counts().to_frame()

# We can see all the primary volcano type values seem to be okay
# We will combine the (es)/(s)/? to their original type

,primaryVolcanoType
Stratovolcano,5254
Stratovolcano(es),1687
Caldera,1085
Shield,1056
Complex,735
Submarine,431
Subglacial,142
Lava dome(s),141
Pyroclastic cone(s),140
Pyroclastic shield,86


In [23]:
# We took the original string , split it to two , first half is until '('
# Then we took the first half , meaning we lost all the character from '(' to the end

for i, j in df.iterrows():
    a_string=j['primaryVolcanoType']
    split_string = str(a_string).split("(", 1)
    substring = split_string[0]
    df.at[i,'primaryVolcanoType']=substring

for i, j in df.iterrows():
    a_string=j['primaryVolcanoType'] 
    split_string = str(a_string).split("?", 1)
    substring = split_string[0]
    df.at[i,'primaryVolcanoType']=substring
    
    
df.primaryVolcanoType.value_counts().to_frame()

,primaryVolcanoType
Stratovolcano,6949
Caldera,1120
Shield,1091
Complex,742
Submarine,431
Lava dome,164
Pyroclastic cone,152
Subglacial,142
Pyroclastic shield,86
nan,77


### <u> Population</u>

In [24]:
df.population5km.value_counts().to_frame()

,population5km
0,2848
78,285
"55,859",197
"75,559",187
477,164
...,...
837,1
970,1
"23,157",1
242,1


In [25]:
df.population10km.value_counts().to_frame()

,population10km
0,2311
"3,291",241
"55,859",197
"75,559",187
"1,067",164
...,...
"49,763",1
"134,114",1
"21,448",1
"80,502",1


In [26]:
df.population30km.value_counts().to_frame()

,population30km
0,1079
"1,016,540",241
"246,792",197
"234,591",187
"35,118",164
...,...
"15,991",1
"195,475",1
"33,262",1
"574,162",1


In [27]:
df.population100km.value_counts().to_frame()

# We can see all the population values seem to be okay

,population100km
0,505
"3,052,770",241
"717,820",197
"6,507,519",187
"667,788",164
...,...
"6,215,853",1
"3,650",1
"259,325",1
"3,001",1


## <font color='#38ADC8'><u>Filling missing values</u></font>



In [28]:
df.isnull().sum()

volcanoName              0
eruptionCertainty        0
startDate                3
maxVEI                2902
evidenceMethod        1409
latitude                77
longitude               77
volcanoHeight           77
country                 77
primaryVolcanoType       0
population5km          108
population10km         122
population30km         122
population100km        122
dtype: int64

### <u> Filling maxVEI with average values</u>

In [29]:
# We created a dictionary that contains all the (VolcanoName : [List of its VEI's])
# Then we created another dictionary that based on VEI's averages and it looks like this (VolcanoName : AverageVEI)

# We used our AVG dictionary to fill in the missing VEI's 

maxVEI_dict = dict()

for i,j in df.iterrows():  
    if (pd.isna(j['maxVEI'])):
        continue
    else:
        if j['volcanoName'] in maxVEI_dict.keys():
            maxVEI_dict[j['volcanoName']].append(j['maxVEI'])
        else:
            maxVEI_dict[j['volcanoName']]=list([j['maxVEI']])          
    
def cal_average(num):
    sum_num = 0
    for t in num:
        sum_num = sum_num + int(t)           

    avg = sum_num / len(num)
    return avg

all_avgs=dict()

for key in maxVEI_dict:
    all_avgs[key]=(int(cal_average(maxVEI_dict[key])))
    
for i, j in df.iterrows():
    if (pd.isna(j['maxVEI']) and j['volcanoName'] in all_avgs):
        df.at[i,'maxVEI']=all_avgs[j['volcanoName']]

  

In [30]:
# Here we remove all the volcanoes who have no maxVEI history at all (no avg)
df=df[~df['maxVEI'].isin([float('NaN')])]

In [31]:
df['maxVEI'].isnull().sum()

0

### <u> Filling Evidence Method with upper neighbour value</u>

In [32]:
df['evidenceMethod'].fillna(method='ffill',inplace=True)

df['evidenceMethod'].isnull().sum()

0

## <font color='#944242'><u> Dropping All the Rows with Missing Values</u></font>




In [33]:
df.isnull().sum()

volcanoName             0
eruptionCertainty       0
startDate               3
maxVEI                  0
evidenceMethod          0
latitude               77
longitude              77
volcanoHeight          77
country                77
primaryVolcanoType      0
population5km         102
population10km        116
population30km        116
population100km       116
dtype: int64

In [34]:
# We can see that only 3 rows remain with missing values so now we will drop them
df.dropna(inplace=True)
df.isnull().sum()

volcanoName           0
eruptionCertainty     0
startDate             0
maxVEI                0
evidenceMethod        0
latitude              0
longitude             0
volcanoHeight         0
country               0
primaryVolcanoType    0
population5km         0
population10km        0
population30km        0
population100km       0
dtype: int64

## <font color='#38ADC8'><u> Converting columns to their appropriate type (Ex. from object to float)</u></font>

In [35]:
df.dtypes

volcanoName            object
eruptionCertainty      object
startDate              object
maxVEI                 object
evidenceMethod         object
latitude              float64
longitude             float64
volcanoHeight          object
country                object
primaryVolcanoType     object
population5km          object
population10km         object
population30km         object
population100km        object
dtype: object

In [36]:
df['eruptionCertainty'].unique()

array(['Confirmed Eruption', 'Uncertain Eruption', 'Discredited Eruption'],
      dtype=object)

In [37]:
df["eruptionCertainty"] = df['eruptionCertainty'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10637 entries, 0 to 11218
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   volcanoName         10637 non-null  object  
 1   eruptionCertainty   10637 non-null  category
 2   startDate           10637 non-null  object  
 3   maxVEI              10637 non-null  object  
 4   evidenceMethod      10637 non-null  object  
 5   latitude            10637 non-null  float64 
 6   longitude           10637 non-null  float64 
 7   volcanoHeight       10637 non-null  object  
 8   country             10637 non-null  object  
 9   primaryVolcanoType  10637 non-null  object  
 10  population5km       10637 non-null  object  
 11  population10km      10637 non-null  object  
 12  population30km      10637 non-null  object  
 13  population100km     10637 non-null  object  
dtypes: category(1), float64(2), object(11)
memory usage: 1.1+ MB


In [38]:
replace_map = {'Confirmed Eruption': 1, 'Uncertain Eruption': 2, 'Discredited Eruption': 3}
df.replace(replace_map, inplace=True)
df.head()

,volcanoName,eruptionCertainty,startDate,maxVEI,evidenceMethod,latitude,longitude,volcanoHeight,country,primaryVolcanoType,population5km,population10km,population30km,population100km
0,Ambae,1,2021 Dec 5,2,Historical Observations,-15.389,167.835,1496 m,Vanuatu,Shield,73,"1,279","4,326","73,308"
1,Pinatubo,1,2021 Nov 30,4,Historical Observations,15.130,120.350,1486 m,Philippines,Stratovolcano,725,"3,808","1,148,684","21,875,048"
2,Iliwerung,1,2021 Nov 28,1,Historical Observations,-8.532,123.573,583 m,Indonesia,Complex,968,"4,318","79,569","665,081"
3,Taal,1,2021 Nov 15,2,Historical Observations,14.002,120.993,311 m,Philippines,Caldera,"717,090","717,090","2,380,326","24,814,047"
4,Turrialba,1,2021 Nov 3,2,Historical Observations,10.025,-83.767,3340 m,Costa Rica,Stratovolcano,"1,891","6,867","672,428","3,390,544"


In [39]:
cols = ['maxVEI', 'latitude', 'longitude', 'population5km', 'population10km', 'population30km', 'population100km', 'volcanoHeight']
problems = ['population5km', 'population10km', 'population30km', 'population100km']
df[problems] = df[problems].replace(',','',regex=True)
df['volcanoHeight'] = df['volcanoHeight'].replace('m','',regex=True)
df[cols] = df[cols].apply(pd.to_numeric)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10637 entries, 0 to 11218
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   volcanoName         10637 non-null  object  
 1   eruptionCertainty   10637 non-null  category
 2   startDate           10637 non-null  object  
 3   maxVEI              10637 non-null  int64   
 4   evidenceMethod      10637 non-null  object  
 5   latitude            10637 non-null  float64 
 6   longitude           10637 non-null  float64 
 7   volcanoHeight       10637 non-null  int64   
 8   country             10637 non-null  object  
 9   primaryVolcanoType  10637 non-null  object  
 10  population5km       10637 non-null  int64   
 11  population10km      10637 non-null  int64   
 12  population30km      10637 non-null  int64   
 13  population100km     10637 non-null  int64   
dtypes: category(1), float64(2), int64(6), object(5)
memory usage: 1.1+ MB


## <font color='#38ADC8'><u> Outliers</u></font>




### <u> Latitude</u>

In [40]:
# Here we are checking if there are outliers in latitude column.
# We know that latitude should be between -90 to 90
print (df.latitude.max())
print (df.latitude.min())

85.608
-77.53


### <u> Longitude</u>

In [41]:
# Here we are checking if there are outliers in latitude column.
# We know that latitude should be between -180 to 180
print (df.longitude.max())
print (df.longitude.min())

179.58
-179.97


### <u> Volcano Height</u>

In [42]:
# We know that the highest volcano in the world is: "Ojos del Salado" and its height is ~ 6800
# We know that the deepest volcano in the world is: "Pico Fracture Zone" and its height is ~ -4200
print(df.volcanoHeight.max())
print(df.volcanoHeight.min())

6879
-5700


In [43]:
df.drop(df.index[df['volcanoHeight'] < (-4200)], inplace=True)
print(df.volcanoHeight.min())

-4200


### <u> Population</u>

In [44]:
# We can see that the max value is correct
print(df.population5km.max())
print(df.population10km.max())
print(df.population30km.max())
print(df.population100km.max())

5783287
5783287
6735396
40640105


In [45]:
# We can see that the min value is 0 which is correct
print(df.population5km.min())
print(df.population10km.min())
print(df.population30km.min())
print(df.population100km.min())

0
0
0
0


<br>

#### <u> After we cleaned the dataframe now we can continue to the EDA part</u>

In [46]:
df.to_csv("Volcano_after_cleaning.csv", index=False, encoding='utf-8')